# TF Serving

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/yggdrasil-decision-forests/blob/main/documentation/public/docs/tutorial/tf_serving.ipynb)

## Setup

In [ ]:
pip install ydf -U

## What is TF Serving?

[TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving) is a production environment for running machine learning models. YDF models are native to TF Serving, meaning that TF Serving natively supports YDF models.

## Train a model

We train a YDF model in a similar fashion to the classification tutorial.

We load a dataset:

In [2]:
import ydf  # Yggdrasil Decision Forests
import pandas as pd  # We use Pandas to load small datasets

# Download a classification dataset and load it as a Pandas DataFrame.
ds_path = "https://raw.githubusercontent.com/google/yggdrasil-decision-forests/main/yggdrasil_decision_forests/test_data/dataset"
train_ds = pd.read_csv(f"{ds_path}/adult_train.csv")
test_ds = pd.read_csv(f"{ds_path}/adult_test.csv")

# Print the first 5 training examples
train_ds.head(5)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,44,Private,228057,7th-8th,4,Married-civ-spouse,Machine-op-inspct,Wife,White,Female,0,0,40,Dominican-Republic,<=50K
1,20,Private,299047,Some-college,10,Never-married,Other-service,Not-in-family,White,Female,0,0,20,United-States,<=50K
2,40,Private,342164,HS-grad,9,Separated,Adm-clerical,Unmarried,White,Female,0,0,37,United-States,<=50K
3,30,Private,361742,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,<=50K
4,67,Self-emp-inc,171564,HS-grad,9,Married-civ-spouse,Prof-specialty,Wife,White,Female,20051,0,30,England,>50K


And we train the model:

In [3]:
model = ydf.GradientBoostedTreesLearner(label="income").train(train_ds)

Train model on 22792 examples


Model trained in 0:00:07.923158


## Export model to TF Saved Model format

Since TensorFlow Serving expects the model in the TF SavedModel format, we export the YDF model to a TF Serving model.

YDF requires [TensorFlow Decision Forests](https://www.tensorflow.org/decision_forests) to be installed to export TF Saved Models. TensorFlow Decision Forests is a Decision Forest library built on top of YDF.

In [ ]:
!pip install tensorflow_decision_forests

In [9]:
model.to_tensorflow_saved_model("/tmp/ydf/my_saved_model")

[INFO 23-11-19 18:42:04.9019 CET kernel.cc:1233] Loading model from path /tmp/tmp8hv3g8sa/ with prefix 
[INFO 23-11-19 18:42:04.9285 CET quick_scorer_extended.cc:903] The binary was compiled without AVX2 support, but your CPU supports it. Enable it for faster model inference.
[INFO 23-11-19 18:42:04.9295 CET abstract_model.cc:1344] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-11-19 18:42:04.9297 CET kernel.cc:1061] Use fast generic engine


TF Serving expects the model path to be in the following format:

`models/<MODEL_NAME>/<VERSION>`


To test our model, we start a local version of TF Serving following the [tf serving setup instructions](https://github.com/tensorflow/serving#set-up).

In a separate terminal, type:

```shell
cd /tmp/ydf

docker run -t --rm -p 8501:8501 \
    -v /tmp/ydf/my_saved_model:/models/my_saved_model/1 \
    -e MODEL_NAME=my_saved_model \
    tensorflow/serving
```

Once the server is up and running, you can send prediction requests. The TF Serving REST API supports two ways to represent examples:

- Instances: This is simpler for when passing a single example.
- Inputs: This is more efficient when running the model on multiple examples.

Here is an example of each:

**Predictions with the instances API:**

In [12]:
!curl http://localhost:8501/v1/models/my_saved_model:predict -X POST \
    -d '{"instances": [{"age":39,"workclass":"State-gov","fnlwgt":77516,"education":"Bachelors","education_num":13,"marital_status":"Never-married","occupation":"Adm-clerical","relationship":"Not-in-family","race":"White","sex":"Male","capital_gain":2174,"capital_loss":0,"hours_per_week":40,"native_country":"United-States"}]}'

{
    "predictions": [[0.0186043456]
    ]
}

**Predictions with the predict+inputs API:**

In [16]:
!curl http://localhost:8501/v1/models/my_saved_model:predict -X POST \
    -d '{"inputs": {"age":[39],"workclass":["State-gov"],"fnlwgt":[77516],"education":["Bachelors"],"education_num":[13],"marital_status":["Never-married"],"occupation":["Adm-clerical"],"relationship":["Not-in-family"],"race":["White"],"sex":["Male"],"capital_gain":[2174],"capital_loss":[0],"hours_per_week":[40],"native_country":["United-States"]} }'

{
    "outputs": [
        [
            0.0186043456
        ]
    ]
}